In [1]:
import os
import ee
ee.Initialize()
import pandas as pd

In [2]:
# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)

# mask out low quality pixels (based on flags)
def maskPixels(image0):
    #Select the QA band
    QA = image0.select('StateQA')
    # Get the land_water_flag bits
    landWaterFlag = getQABits(QA, 3, 5, 'land_water_flag')
    #Get the cloud_state bits and find cloudy areas.
    cloud = getQABits(QA, 0, 1, 'cloud_state').expression("b(0) == 1 || b(0) == 2")
    # Get the cloud_shadow bit
    cloudShadows = getQABits(QA, 2, 2, 'cloud_shadow')
    # Get the Pixel is adjacent to cloud bit
    cloudAdjacent = getQABits(QA, 13, 13, 'cloud_adj')
    # Get the internal cloud flag
    cloud2 = getQABits(QA, 10, 10, 'cloud_internal')
    # Get the internal fire flag
    fire = getQABits(QA, 11, 11, 'fire_internal')
    # Get the MOD35 snow/ice flag
    snow1 = getQABits(QA, 12, 12, 'snow_MOD35')
    # Get the internal snow flag
    snow2 = getQABits(QA, 15, 15, 'snow_internal')
    # create mask
    mask = landWaterFlag.eq(1).And(cloud.Not()).And(cloudShadows.Not()).And(cloudAdjacent.Not()).And(cloud2.Not()).And(fire.Not()).And(snow1.Not()).And(snow2.Not())
    return image0.updateMask(mask) 

In [55]:
collection = ee.ImageCollection('MODIS/006/MOD09A1').filterDate('2000-01-01', '2020-12-31')

In [71]:
start = 3
end = 5
pattern = 0
for i in range(start,end+1):
    print(i,'original value')
    pattern += 2**i
    print(2**i,'part on the right hand side')
print(pattern,'pattern')

3 original value
8 part on the right hand side
4 original value
16 part on the right hand side
5 original value
32 part on the right hand side
56 pattern


In [56]:
image = collection.first()

In [58]:
QA = image.select('StateQA')
# Get the land_water_flag bits
landWaterFlag = getQABits(QA, 3, 5, 'land_water_flag')

In [60]:
import geemap
Map = geemap.Map()

In [73]:
myimg = image.select('StateQA').select([0],['land_water_flag']).bitwiseAnd(56).rightShift(3).eq(1)


In [64]:
Map.addLayer(mask)

In [62]:
Map.addLayer(landWaterFlag)

In [79]:
Map.addLayer(QA,'','QA')

In [80]:
Map.addLayer(myimg,'','LAND')

In [61]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [82]:
1 << 10

1024

In [83]:
bin(10)

'0b1010'